In [12]:
import pandas as pd
import glob
import os
from datetime import datetime

folder_path = '文本数据'
name = '家电'

# 匹配所有 *食品饮料*.csv 文件
file_pattern = os.path.join(folder_path, f'*{name}*.csv')
file_list = glob.glob(file_pattern)

if not file_list:
    print("未找到匹配的文件！")
else:
    # 指定需要保留的列
    required_columns = ['author', 'comments', 'update']
    
    df_list = []
    for file in file_list:
        print(f"正在处理文件: {file}")
        df = pd.read_csv(file)
        cols_to_keep = [col for col in required_columns if col in df.columns]
        df = df[cols_to_keep]
        df_list.append(df)
    
    # 合并
    merged_df = pd.concat(df_list, ignore_index=True)

    if 'update' not in merged_df.columns:
        raise ValueError("所有文件中都缺少 'update' 列！")

    # 补全年份以便解析日期（假设都是当前年）
    current_year = datetime.now().year
    merged_df['update_full'] = merged_df['update'].astype(str).apply(
        lambda x: f"{current_year}-{x}" if '-' in x and len(x.split('-')[0]) <= 2 else x
    )

    # 转为 datetime
    merged_df['update_date'] = pd.to_datetime(merged_df['update_full'], format='%Y-%m-%d', errors='coerce')
    merged_df = merged_df.dropna(subset=['update_date'])

    # === 新增：筛选 05-01 到 10-31 的数据（不依赖年份）===
    # 提取月和日组成 MM-DD 字符串
    merged_df['mm_dd'] = merged_df['update_date'].dt.strftime('%m-%d')
    
    # 定义起止
    start = '05-01'
    end = '10-31'
    
    # 筛选：注意这里直接字符串比较是安全的，因为格式固定为 mm-dd
    filtered_df = merged_df[(merged_df['mm_dd'] >= start) & (merged_df['mm_dd'] <= end)].copy()

    # 按 update_date 排序
    filtered_df = filtered_df.sort_values(by='update_date', ascending=False).reset_index(drop=True)

    # 只保留原始列
    final_df = filtered_df[required_columns]

    # 保存结果
    output_path = os.path.join("文本数据合并", f'merged_{name}.csv')
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"成功合并 {len(file_list)} 个文件，筛选后共 {len(final_df)} 行数据（05-01 至 10-31），已按 'update' 排序。")

正在处理文件: 文本数据\【家电】bk0456_page-1-4.csv
正在处理文件: 文本数据\【家电（美的）】000333_page-27-215_converted.csv
成功合并 2 个文件，筛选后共 14151 行数据（05-01 至 10-31），已按 'update' 排序。


In [13]:
import pandas as pd
name = '家电' # 动漫，家电，旅游，汽车，食品饮料，文娱传媒
# 读取数据
df = pd.read_csv(f'文本数据合并/merged_{name}.csv')

# 可选：确保只包含必要列（根据你展示的数据）
df = df[['author', 'comments', 'update']].copy()

# 按 update 分组，每组取最多前 3 行
df_limited = df.groupby('update').head(3).reset_index(drop=True)

# 保存结果（可选）
df_limited.to_csv(f'文本数据合并/merged_{name}_top3_per_day.csv', index=False, encoding='utf-8-sig')

print(f"处理完成！原始 {len(df)} 行，筛选后 {len(df_limited)} 行。")
df_limited.head(10)

处理完成！原始 14151 行，筛选后 552 行。


,author,comments,update
0,n1652137528020204934,今天出一部分做个T？能否，老师们,10-31
1,股海垂钓趋势投资,科技回落，转战低估红利,10-31
2,wqxiangfei,被这垃圾股套好多年，91买的,10-31
3,股友2369Y861v9,调查下，谁用过小米空调啊，真的能制冷么，是不是只是个只吹风的空壳啊,10-30
4,A股20年的巴菲特,工业互联一家台湾公司，市值1.5万亿，动不动就涨停，动不动就涨五六个点，你才五六,10-30
5,开心就好178,美的开始第二波主升浪，奥利给,10-30
6,股友2369Y861v9,这样的大盘走势美的不涨，那么大盘下行的时候还会回到72附近,10-29
7,志远商贸有限公司,硬要亏损一分,10-29
8,阳光的许哲,恶心，真是恶心，都不能红一下,10-29
9,闲云野鹤wws,美的另有担当，护大盘，是股指期货标的又要兼顾H股，真正反映家电块趋势的是格力与海,10-28


In [11]:
import pandas as pd
import re
df = pd.read_csv('文本数据/【家电（美的）】000333_page-27-215.csv')

def to_mmdd(date_str):
    match = re.match(r'(\d{1,2})月(\d{1,2})日', date_str)
    if match:
        return f"{int(match.group(1)):02d}-{int(match.group(2)):02d}"
    return None

df['update'] = df['update'].apply(to_mmdd)
df.to_csv('文本数据/【家电（美的）】000333_page-27-215_converted.csv', index=False, encoding='utf-8-sig')

In [3]:
import pandas as pd
from pathlib import Path

# 设置输入输出目录
input_dir = Path("output")
output_dir = input_dir  # 也可以设为其他目录，如 Path("output/daily_avg")

# 确保输出目录存在（如果另设路径）
output_dir.mkdir(exist_ok=True)

# 查找所有 _sentiment.csv 文件
sentiment_files = list(input_dir.glob("*_sentiment.csv"))

if not sentiment_files:
    print("⚠️ 未找到任何 *_sentiment.csv 文件")
else:
    print(f"📂 找到 {len(sentiment_files)} 个情感分析结果文件，开始处理...")

for file_path in sentiment_files:
    try:
        print(f"📄 处理: {file_path.name}")
        
        # 读取文件
        df = pd.read_csv(file_path, encoding='utf-8-sig')
        
        # 检查必要列是否存在
        if 'update' not in df.columns or 'sentiment_score' not in df.columns:
            print(f"  ⚠️ 跳过 {file_path.name}：缺少 'update' 或 'sentiment_score' 列")
            continue

        # 确保 update 是字符串
        df['update'] = df['update'].astype(str)
        
        # 按 update 分组求平均
        daily_avg = df.groupby('update')['sentiment_score'].mean().round(2).reset_index()
        
        # （可选）按日期排序（假设是同一年内的 MM-DD 格式）
        # 如果跨年，建议补充年份后再排序
        daily_avg = daily_avg.sort_values('update').reset_index(drop=True)
        
        # 构造新文件名：原名 + _daily_avg.csv
        new_filename = file_path.stem + "_daily_avg.csv"
        output_file = output_dir / new_filename
        
        # 保存
        daily_avg.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"  ✅ 已保存: {output_file.name}")

    except Exception as e:
        print(f"  ❌ 处理 {file_path.name} 时出错: {e}")

print("\n🎉 所有文件的日均情感分已生成完毕！")

📂 找到 6 个情感分析结果文件，开始处理...
📄 处理: merged_动漫_top3_per_day_sentiment.csv
  ✅ 已保存: merged_动漫_top3_per_day_sentiment_daily_avg.csv
📄 处理: merged_家电_top3_per_day_sentiment.csv
  ✅ 已保存: merged_家电_top3_per_day_sentiment_daily_avg.csv
📄 处理: merged_文娱传媒_top3_per_day_sentiment.csv
  ✅ 已保存: merged_文娱传媒_top3_per_day_sentiment_daily_avg.csv
📄 处理: merged_旅游_top3_per_day_sentiment.csv
  ✅ 已保存: merged_旅游_top3_per_day_sentiment_daily_avg.csv
📄 处理: merged_汽车_top3_per_day_sentiment.csv
  ✅ 已保存: merged_汽车_top3_per_day_sentiment_daily_avg.csv
📄 处理: merged_食品饮料_top3_per_day_sentiment.csv
  ✅ 已保存: merged_食品饮料_top3_per_day_sentiment_daily_avg.csv

🎉 所有文件的日均情感分已生成完毕！
